# Metaprogramming 

## Putting a wrapper around a function.

### Problem: You want to put a wrapper layer around a function that adds extra processing (e.g., logging, timing, etc.).

In [1]:
# Define a decorator function.
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end - start)
        return result
    return wrapper

In [2]:
@timethis
def countdown(n):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1

In [3]:
countdown(100_000)

countdown 0.018427133560180664


In [4]:
countdown(100_000_000)

countdown 3.2224884033203125


#### Decorator
A decorator is a function that accepts a function as input and returns a new function as output.

```
@timethis
def countdown(n):
    ...
```
is same as:

```
def countdown(n):
    ...
    
countdown = timethis(countdown)
```

The code inside a decorator typically involves creating a new function that accepts any arguments using *args and **kwargs.

## Preserving Function Metadata when Writing Decorators

### Problem: You've written a decorator, but when you apply it to a function, important metadata such as the name, doc string, annotations, and calling signature are lost. 

In [5]:
# Always remember to apply the @wraps decorator from functools.
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end - start)
        return result
    return wrapper

In [6]:
@timethis
def countdown(n:int):
    '''
    Counts down.
    '''
    while n > 0:
        n -= 1

In [7]:
countdown(100_000)

countdown 0.006591320037841797


Accessing the metadata. This is possible due to the use of @wraps decorator.

In [8]:
countdown.__name__

'countdown'

In [9]:
countdown.__doc__

'\n    Counts down.\n    '

In [10]:
countdown.__annotations__

{'n': int}

@wraps makes the wrapped function available to you in the ```__wrapped__``` attribute.

In [11]:
countdown.__wrapped__(100_000)

```__wrapped__``` attribute also makes decorated functions properly expose the underlying signature of the wrapped function.

In [12]:
from inspect import signature
print(signature(countdown))

(n: int)


## Unwrapping a Decorator

### Problem: A decorator has been applied to a function, but you want to "undo" it, gaining access to the original unwrapped function.

In [17]:
## Assuming the decorator was properly implemented using @wraps
@timethis  # somedecorator
def add(x, y):
    return x + y

orig_add = add.__wrapped__

In [18]:
orig_add(3, 4)

7

Gaining direct access to the unwrapped function behind a decorator can be useful for debuggin, introspection, and other operations involving functions. But it's only applicable if the decorator was implemented using the @wraps decorator.

#### Multiple decorators
```__wrapped__``` is undefined and should not be used in case of multiple decorators.

In [19]:
from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper

In [20]:
def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

In [21]:
@decorator1
@decorator2
def add(x, y):
    return x + y